In [21]:
import textstat

#text = "Graphics library to create an embedded GUI with easy-to-use graphical elements, beautiful visual effects and low memory footprint. It offers anti-aliasing, opacity, and animations using only one frame buffer. "
#text = "Alternative firmware for ESP8266 and ESP32 based devices with easy configuration using webUI, OTA updates, automation using timers or rules, expandability and entirely local control over MQTT, HTTP, Serial or KNX. Full documentation at"
text = "my exp for CVE-2020-27194, tested on linux kernel 5.8.14."
#text = "🎬 IMDb: Link 'em all!  -  Adds all kinds of links to IMDb, customizable!"
#text = "A sexy achievement file parser with real-time notification, automatic screenshot and playtime tracking. View every achievements earned on your PC whether it's coming from Steam, a Steam emulator, and more."
#text = "Complete-Python-3-Bootcamp"

In [5]:
d = {"github:fei4xu": [{"tokens": ["meta", "tegra", "bsp", "layer", "nvidia", "jetson", "platform", "based", "lt"], "tags": ["github:OE4T/meta-tegra", "meta-tegra", "BitBake"]}, {"tokens": ["rdparty", "rd", "party", "dependency", "required", "skui"], "tags": ["github:skui-org/3rdparty", "3rdparty", "C++"]}, {"tokens": ["skui", "skia", "based", "ui", "framework"], "tags": ["github:skui-org/skui", "skui", "C++", "cmake", "cross-platform", "cxx", "cxx17", "graphics", "gui", "opengl", "signal", "skia", "ui"]}, {"tokens": ["skia", "cmake", "skia", "fork", "cmake"], "tags": ["github:fei4xu/Skia-cmake", "Skia-cmake", "CMake"]}], "github:sergiotapia": [{"tokens": ["torrentinim", "low", "memory", "footprint", "self", "hosted", "api", "torrent", "search", "engine", "sonarr", "radarr", "compatible", "native", "support", "linux", "mac", "window"], "tags": ["github:sergiotapia/torrentinim", "torrentinim", "Nim", "bittorrent", "p2p", "radarr", "search-engine", "self-hosted", "sonarr", "torrent", "torrent-search-engine"]}, {"tokens": ["magnetissimo", "web", "application", "index", "popular", "torrent", "site", "save", "local", "database"], "tags": ["github:sergiotapia/magnetissimo", "magnetissimo", "Elixir", "bittorrent", "p2p", "search-engine", "self-hosted", "torrent"]}, {"tokens": ["golden", "sample", "phoenix", "liveview", "application", "using", "esbuild", "alpinejs", "tailwind", "jit"], "tags": ["github:sergiotapia/golden", "golden", "Elixir"]}, {"tokens": ["lostinthegc", "com", "hugo", "blog", "website"], "tags": ["github:sergiotapia/lostinthegc.com", "lostinthegc.com", "CSS"]}, {"tokens": ["filesize", "nim", "package", "convert", "filesizes", "unit", "turn", "filesizes", "human", "readable", "string"], "tags": ["github:sergiotapia/filesize", "filesize", "Nim"]}], "github:Riya-code": [{"tokens": ["eqequation", "website", "gssoc", "extended", "website", "emotional", "intelligence", "mental", "health"], "tags": ["github:girlscript/EQEquation-Website", "EQEquation-Website", "HTML", "eqequation-website", "gssoc", "hacktoberfest", "hacktoberfest2020"]}, {"tokens": ["git", "basic", "pwa", "twa", "git", "github", "concept", "command"], "tags": ["github:HarshKapadia2/git_basics", "git_basics", "JavaScript", "asciidoctor", "asciidoctor-jet", "git", "github", "gitpod", "hacktoberfest", "pwa", "twa"]}, {"tokens": ["passgen", "cpp", "cli", "password", "generator", "written"], "tags": ["github:hnccbits/passgen-cpp", "passgen-cpp", "C++"]}, {"tokens": ["engineering", "note", "collection", "resource", "section", "student"], "tags": ["github:tewarig/Engineering-notes", "Engineering-notes", "HTML", "html", "notes", "students", "study"]}, {"tokens": ["jekyll", "material", "theme", "jekyll", "theme", "based", "material", "design", "using", "materialize"], "tags": ["github:jameshamann/jekyll-material-theme", "jekyll-material-theme", "HTML", "jekyll-site", "jekyll-themes", "material", "material-design", "material-theme", "material-ui", "materialize", "materialize-css", "materializecss"]}, {"tokens": ["riya", "code", "github", "io", "jekyll", "theme", "based", "material", "design", "using", "materialize"], "tags": ["github:Riya-code/Riya-code.github.io", "Riya-code.github.io", "HTML"]}, {"tokens": ["passgen", "cpp", "cli", "password", "generator", "written"], "tags": ["github:Riya-code/passgen-cpp", "passgen-cpp", "CMake"]}], "github:ZijianDu": [{"tokens": ["awesome", "deblurring", "curated", "list", "resource", "image", "video", "deblurring"], "tags": ["github:ZijianDu/Awesome-Deblurring", "Awesome-Deblurring", "Jupyter Notebook"]}, {"tokens": ["design", "pattern", "design", "pattern", "aiming", "resuable", "object", "oriented", "design"], "tags": ["github:ZijianDu/Design_pattern", "Design_pattern", "C++"]}, {"tokens": ["selfdeblur", "neural", "blind", "deconvolution", "using", "deep", "prior", "cvpr"], "tags": ["github:ZijianDu/SelfDeblur", "SelfDeblur", "Python"]}], "github:Obamium37": [{"tokens": ["himcm", "model", "arima", "model", "built", "predicting", "future", "level", "built", "comap", "himcm", "competition"], "tags": ["github:Obamium37/ARIMA-Time-Forecasting-Model-for-HIMCM", "2022-HIMCM-Models", "Jupyter Notebook"]}, {"tokens": ["cloud", "notetaker", "basic", "weekend", "project", "cloud", "notetaker", "made", "using", "reactjs", "library", "firebase", "firestore"], "tags": ["github:Obamium37/Cloud-Notetaker", "Cloud-Notetaker", "JavaScript"]}, {"tokens": ["service", "hour", "logger", "service", "hour", "logger", "us", "reactjs", "firebase", "firestore", "log", "community", "service", "hour", "export", "excel", "spreadsheet"], "tags": ["github:Obamium37/Service-Hours-Logger", "Service-Hours-Logger", "JavaScript"]}], "github:blueneutron": [{"tokens": ["android", "testing", "sample", "app", "demonstrate", "android", "unit", "ui", "testing"], "tags": ["github:blueneutron/Android-Testing", "android-testing", "Java", "android", "espresso", "mvp-android", "ui-testing", "unit-testing"]}], "github:shashi7679": [{"tokens": ["birdclassifier", "classifies", "bird", "class"], "tags": ["github:shashi7679/BirdClassifier", "BirdClassifier", "Jupyter Notebook", "cnn", "cnn-classification", "pytorch", "resnet-50", "streamlit"]}, {"tokens": ["covid", "lung", "radiography", "classifying", "image", "chest", "ray", "scan", "using", "cnn"], "tags": ["github:shashi7679/covid-19-lungs-radiography", "covid-19-lungs-radiography", "Jupyter Notebook", "covid19-data", "fastapi", "image-classification", "tensorflow"]}, {"tokens": ["self", "driving", "car", "raspberrypi", "self", "driving", "car", "using", "image", "processing"], "tags": ["github:shashi7679/self-driving-car-raspberrypi", "self-driving-car-raspberrypi", "C++", "autonomous-vehicles", "computer-vision", "opencv-python", "raspberry-pi-3"]}, {"tokens": ["pixpix", "gans", "generate", "map", "using", "satellite", "image"], "tags": ["github:shashi7679/pix2pix-GANs", "pix2pix-GANs", "Jupyter Notebook", "deep-learning", "gan", "generative-adversarial-network", "image-to-image-translation", "pytorch"]}, {"tokens": ["cyclegan", "implementation", "cycle", "gan", "pytorch"], "tags": ["github:shashi7679/CycleGAN", "CycleGAN", "Python"]}, {"tokens": ["shashi", "config", "file", "github", "profile"], "tags": ["github:shashi7679/shashi7679", "shashi7679", "config", "github-config"]}, {"tokens": ["basicsofprogramming", "basic", "programming", "problem", "solution"], "tags": ["github:shashi7679/BasicsOfProgramming", "BasicsOfProgramming", "C++"]}, {"tokens": ["plantvillage", "cnn", "end", "end", "image", "classification", "using", "cnn", "plantvillage", "dataset"], "tags": ["github:shashi7679/PlantVillage-cnn", "PlantVillage-cnn", "Python", "cnn", "fastapi", "image-classification", "plantvillage-dataset"]}], "github:siinestesiia": [{"tokens": ["siinestesiia", "config", "file", "github", "profile"], "tags": ["github:siinestesiia/siinestesiia", "siinestesiia", "config", "github-config"]}], "github:brunohomma": [{"tokens": ["awesome", "python", "script", "curated", "collection", "heart", "eye", "cool", "python", "script"], "tags": ["github:ankitdobhal/Awesome-Python-Scripts", "Awesome-Python-Scripts", "Python", "automation", "beginner-project", "ch20", "hacktoberfest", "open-source", "python", "python-scripts"]}], "github:NirobNabil": [{"tokens": ["drabdulmannan", "jamstack", "site", "created", "stackbit"], "tags": ["github:NirobNabil/magenta-sun-80a90", "drabdulmannan", "JavaScript", "gatsby", "git", "headless", "jamstack", "ssg", "stackbit", "static"]}, {"tokens": ["drabdulmannan", "jamstack", "site", "created", "stackbit"], "tags": ["github:NirobNabil/drabdulmannan", "drabdulmannan", "JavaScript", "gatsby", "git", "headless", "jamstack", "ssg", "stackbit", "static"]}, {"tokens": ["cf", "activity", "gamma", "checking", "codeforces", "api"], "tags": ["github:Bhrom-but-better/CF_Activity_Gamma", "CF_Activity_Gamma", "Python"]}, {"tokens": ["usv", "sim", "lsa", "unmanned", "surface", "vehicle", "simulation", "gazebo", "water", "current", "wind"], "tags": ["github:NirobNabil/usv_sim_lsa", "usv_sim_lsa", "Python"]}, {"tokens": ["usv", "sim", "lsa", "unmanned", "surface", "vehicle", "simulation", "gazebo", "water", "current", "wind"], "tags": ["github:disaster-robotics-proalertas/usv_sim_lsa", "usv_sim_lsa", "Python", "boat", "disaster-response", "environmental-modelling", "gazebo", "hec-ras", "openfoam", "robotics-simulation", "ros-kinetic", "sailboat", "unmanned-surface-vehicle", "usv", "usv-sim", "usv-simulator", "water-simulation", "water-surface"]}, {"tokens": ["lfr", "code", "lfr", "using", "ir", "sensor"], "tags": ["github:NirobNabil/LFR", "LFR", "C++"]}], "github:mdrizzle": [{"tokens": ["cpp", "pratice", "pracice", "primer", "plus"], "tags": ["github:mdrizzle/Cpp_pratice", "Cpp_pratice", "C++"]}], "github:Ramachetan": [{"tokens": ["first", "contribution", "help", "beginner", "contribute", "open", "source", "project"], "tags": ["github:firstcontributions/first-contributions", "first-contributions", "beginner", "beginner-friendly", "community", "contribution", "contributions", "contributions-welcome", "good-first-issue", "help-wanted", "open-source", "tutorial", "tutorials"]}, {"tokens": ["medium", "ip", "repository", "code", "made", "ihlp", "final", "project"], "tags": ["github:Ramachetan/IHLP-Final-Project", "Media-over-IP", "JavaScript"]}, {"tokens": ["vehicle", "text", "recognition", "aws", "version", "control", "cloud", "computing", "course", "programming", "assignment"], "tags": ["github:Ramachetan/CS-643-PA-1", "Vehicle-Text-Recognition-over-AWS", "Java"]}, {"tokens": ["python", "programming", "python", "programming", "lab", "jntuk"], "tags": ["github:Ramachetan/Python-Programming", "Python-Programming", "Python"]}, {"tokens": ["virtual", "self", "driving", "car", "using", "python", "simple", "virtual", "self", "driving", "car", "build", "using", "ananconda"], "tags": ["github:Ramachetan/Virtual-Self-Driving-Car-Using-Python", "Virtual-Self-Driving-Car-Using-Python", "Python"]}, {"tokens": ["augmented", "reality", "app", "using", "unity", "simple", "ar", "app", "show", "projection", "wissenaire", "logo", "target", "image"], "tags": ["github:Ramachetan/Augmented-Reality-app-using-Unity-3D", "Augmented-Reality-app-using-Unity-3D", "Jupyter Notebook"]}, {"tokens": ["wireless", "usb", "repositry", "wireless", "usb", "project", "made", "ece", "project"], "tags": ["github:Ramachetan/ECE-692-Project", "Wireless_USB", "C++"]}], "github:asgaraliyev": [{"tokens": ["three", "vite", "created", "stackblitz"], "tags": ["github:asgaraliyev/three-vite", "three-vite", "JavaScript"]}, {"tokens": ["express", "typescript", "starter", "kit", "created", "stackblitz"], "tags": ["github:asgaraliyev/express-typescript-starter-kit", "express-typescript-starter-kit", "TypeScript"]}, {"tokens": ["nextjs", "rem", "created", "stackblitz"], "tags": ["github:asgaraliyev/nextjs-rem", "nextjs-rem", "HTML"]}, {"tokens": ["chat", "app", "chat", "app", "meteor", "app"], "tags": ["github:asgaraliyev/chat_app", "chat_app", "JavaScript"]}, {"tokens": ["todo", "app", "simple", "todo", "app", "meteor"], "tags": ["github:asgaraliyev/todo-app", "todo-app", "SCSS"]}, {"tokens": ["react", "router", "starter", "created", "stackblitz"], "tags": ["github:asgaraliyev/react-18-router-starter", "react-18-router-starter", "JavaScript"]}, {"tokens": ["react", "learning", "created", "stackblitz"], "tags": ["github:asgaraliyev/react-18-learning-1", "react-18-learning-1", "TypeScript"]}, {"tokens": ["asgaraliyev", "config", "file", "github", "profile"], "tags": ["github:asgaraliyev/asgaraliyev", "asgaraliyev", "config", "github-config"]}, {"tokens": ["react", "redux", "starter", "kit", "react", "redux", "starter", "kit"], "tags": ["github:asgaraliyev/react-redux-starter-kit", "react-redux-starter-kit", "JavaScript"]}, {"tokens": ["vitejs", "vite", "xschbr", "created", "stackblitz"], "tags": ["github:asgaraliyev/vitejs-vite-xschbr", "vitejs-vite-xschbr", "CSS"]}], "github:GizemnurTaskin": [{"tokens": ["nodejs", "project", "several", "javascript", "project", "using", "nodejs"], "tags": ["github:GizemnurTaskin/NodeJS-Projects", "NodeJS-Projects", "JavaScript"]}], "github:lovegap": [{"tokens": ["nyan", "cat", "template", "responsive", "nyan", "cat", "template"], "tags": ["github:lovegap/nyan-cat-template", "nyan-cat-template", "CSS", "cat", "css", "homepage", "nyan", "nyancat", "responsive", "template"]}], "github:Massas": [{"tokens": ["repository", "study", "html", "cs", "javascript", "amazon", "node", "j"], "tags": ["github:Massas/http-css", "http-css", "HTML", "http-css", "nodejs", "s3-storage"]}, {"tokens": ["algorithm", "repository", "algorithm", "learning"], "tags": ["github:Massas/Clang_Algorithm", "C_Algorithm", "C", "algorithm", "c"]}, {"tokens": ["developer", "community", "stats", "repository", "encourage", "beginner", "contribute", "open", "source", "contributor", "view", "github", "stats"], "tags": ["github:subeshb1/developer-community-stats", "developer-community-stats", "JavaScript", "beginner-friendly", "community", "developer", "developer-community", "gatsby", "github", "good-first-issue", "open-source", "react"]}, {"tokens": ["first", "contribution", "help", "beginner", "contribute", "open", "source", "project"], "tags": ["github:firstcontributions/first-contributions", "first-contributions", "beginner", "beginner-friendly", "community", "contribution", "contributions", "contributions-welcome", "good-first-issue", "help-wanted", "open-source", "tutorial", "tutorials"]}, {"tokens": ["hackerrank", "bash", "solved", "problem", "hackerrank", "bash"], "tags": ["github:Massas/HackerRank_bash", "HackerRank_bash", "Shell", "bash", "hackerrank", "shell"]}, {"tokens": ["winform", "graphic", "powershell", "using", "graphic", "class", "powershell"], "tags": ["github:Massas/WinForm_Graphics_PowerShell", "WinForm_Graphics_PowerShell", "PowerShell"]}, {"tokens": ["jruby", "sample", "jruby", "sample"], "tags": ["github:Massas/jruby_Sample", "jruby_Sample", "Ruby"]}, {"tokens": ["deepl", "py", "python", "wrapper", "deepl", "api"], "tags": ["github:Massas/deepl.py", "deepl.py", "Python"]}, {"tokens": ["imageprocessing", "powershell", "image", "processing", "powershell", "script", "sample"], "tags": ["github:Massas/ImageProcessing_powershell", "ImageProcessing_powershell", "PowerShell"]}, {"tokens": ["footballteamnamemaker", "tool", "fictional", "soccer", "team", "name", "fc", "melrose", "sc", "monteforte", "irpino", "united"], "tags": ["github:Massas/FootballTeamNameMaker", "FootballTeamNameMaker", "PowerShell", "football", "googlemaps-api", "powershell-script", "soccer", "tools"]}, {"tokens": ["hackerrank", "solved", "problem", "hackerrank"], "tags": ["github:Massas/HackerRank_C", "HackerRank_C", "C", "algorithm", "c", "hackerrank"]}, {"tokens": ["powershell", "eng", "kana", "powershell", "script", "convert", "english", "katakana"], "tags": ["github:Massas/powershell_eng-kana", "powershell_eng-kana", "PowerShell", "conversion", "italy-data", "japanese-kana", "powershell"]}, {"tokens": ["hackarrank", "sql", "solved", "problem", "hackerrank", "sql"], "tags": ["github:Massas/HackarRank_SQL", "HackarRank_SQL", "db2luw", "hackerrank", "oracle", "sql"]}, {"tokens": ["shellscript", "various", "shellscripts", "written", "ksh"], "tags": ["github:Massas/shellscript", "shellscript", "Shell", "db2", "db2luw", "ksh", "shellscript"]}, {"tokens": ["t", "sample", "typescript", "sample"], "tags": ["github:Massas/ts-sample1", "ts-sample1", "TypeScript"]}, {"tokens": ["dockersample", "docker", "window", "wsl", "sample"], "tags": ["github:Massas/dockerSample", "dockerSample", "PHP"]}], "github:20011108": [{"tokens": ["config", "file", "github", "profile"], "tags": ["github:20011108/20011108", "20011108", "C++", "config", "github-config"]}], "github:SudoSmart": [{"tokens": ["machinelearningmodels", "repo", "contains", "machine", "learning", "model", "implemented", "tensorflow"], "tags": ["github:SudoSmart/MachineLearningModels", "MachineLearningModels", "Jupyter Notebook", "hacktoberfest"]}], "github:TotallyBiased": [{"tokens": ["promise", "trampoline", "motivating", "example", "javascript", "feature", "way", "thunk", "ish", "promise", "trampoline", "also", "known", "continuation", "passing", "style", "control", "flow"], "tags": ["github:TotallyBiased/promise-trampoline", "promise-trampoline", "JavaScript"]}], "github:JakubKonert": [{"tokens": ["arduino", "dogo", "project", "made", "using", "integrated", "circuit", "arduino"], "tags": ["github:JakubKonert/Arduino_Dogo", "Arduino_Dogo", "C++"]}, {"tokens": ["python", "oczko", "python", "game", "oczko", "sth", "like", "blackjack"], "tags": ["github:JakubKonert/Python_Oczko", "Python_Oczko", "Python"]}, {"tokens": ["website", "polling", "assessment", "addiction", "computer", "game", "among", "student", "website", "task", "collect", "data", "predict", "whether", "person", "addicted", "display", "final", "result", "based", "respondent", "response"], "tags": ["github:drachu/website_polling", "website_polling", "HTML"]}], "github:realStrixxy": [{"tokens": ["htmlpasswordgen", "generates", "random", "password", "specified", "length", "choose", "alphanumerical", "non", "alphanumerical"], "tags": ["github:realStrixxy/HtmlPasswordGen", "HtmlPasswordGen", "JavaScript"]}], "github:Pulkit3108": [{"tokens": ["codeforces", "problem", "solution", "codeforces", "problem"], "tags": ["github:Pulkit3108/Codeforces-Problems", "Codeforces-Problems", "Python"]}, {"tokens": ["minor", "project", "repository", "contains", "minor", "project"], "tags": ["github:Pulkit3108/Minor-Projects", "Minor-Projects", "Java"]}], "github:NikolayCheremnov": [{"tokens": ["vlesbook", "backend", "velesov", "book", "falsification", "created", "th", "xx", "century", "primitively", "imitating", "pre", "slavic", "language"], "tags": ["github:cochegarka/vlesbook-backend", "vlesbook-backend", "Go"]}, {"tokens": ["matfl", "lab", "draft", "laboratory", "work", "discipline", "method", "analysis", "theory", "formal", "language"], "tags": ["github:NikolayCheremnov/matfl_labs", "matfl_labs", "Go"]}, {"tokens": ["constructprojects", "kid", "construct", "project", "deploy"], "tags": ["github:TOPAcademyBarnaul/ConstructProjects", "ConstructProjects", "JavaScript"]}, {"tokens": ["lr", "cg", "cheremnov", "computer", "graphic", "lr"], "tags": ["github:NikolayCheremnov/lr1_CG_Cheremnov", "lr1_CG_Cheremnov", "C#"]}, {"tokens": ["shump", "example", "shump", "genre", "game", "star", "war", "pygame"], "tags": ["github:NikolayCheremnov/shump", "shump", "Python"]}], "github:mholzel": [{"tokens": ["ouster", "sdk", "ouster", "inc", "sample", "code"], "tags": ["github:ouster-lidar/ouster_example", "ouster-sdk", "C++"]}], "github:mrehanali127": [{"tokens": ["hand", "prolog", "problem", "solved", "using", "prolog", "use", "pyswip", "creative", "readable", "user", "interface"], "tags": ["github:mrehanali127/Hands-On-Prolog", "Hands-On-Prolog", "Python"]}, {"tokens": ["searching", "movie", "separate", "chaining", "separate", "chaining", "data", "structure", "project"], "tags": ["github:mrehanali127/Searching-Movies-by-Separate-Chaining", "Searching-Movies-by-Separate-Chaining", "C++"]}, {"tokens": ["snail", "game", "using", "minimax", "algorithm", "two", "player", "game", "human", "v", "bot", "python"], "tags": ["github:mrehanali127/Snails-Game-using-Minimax-Algorithm", "Snails-Game-using-Minimax-Algorithm", "HTML"]}, {"tokens": ["meal", "app", "meal", "app", "dummy", "data"], "tags": ["github:mrehanali127/Meal-App_V1", "Meal-App_V1", "JavaScript"]}], "github:mwoehlke-kitware": [{"tokens": ["seal", "tk", "seal", "tk", "stereoscopic", "examination", "aquatic", "life", "tookit"], "tags": ["github:Kitware/seal-tk", "seal-tk", "C++"]}, {"tokens": ["pip", "drake", "work", "progress", "pypi", "pip", "packaging", "configuration", "drake"], "tags": ["github:RobotLocomotion/pip-drake", "pip-drake", "Makefile", "drake", "pip", "robotics"]}, {"tokens": ["drake", "ci", "continuous", "integration", "script", "drake"], "tags": ["github:RobotLocomotion/drake-ci", "drake-ci", "CMake", "continuous-integration", "drake", "robotics"]}, {"tokens": ["krest", "library", "creating", "gui", "application", "use", "kwiver"], "tags": ["github:Kitware/krest", "krest", "C++"]}, {"tokens": ["qtextensions", "open", "source", "collection", "utility", "class", "tool", "qt"], "tags": ["github:Kitware/qtextensions", "qtextensions", "C++"]}, {"tokens": ["pybind", "seamless", "operability", "python"], "tags": ["github:pybind/pybind11", "pybind11", "C++", "bindings", "python"]}, {"tokens": ["valhalla", "open", "source", "routing", "engine", "openstreetmap"], "tags": ["github:valhalla/valhalla", "valhalla", "C++", "astar", "dijkstra", "directions", "isochrones", "multi-modal", "openstreetmap", "routing", "routing-engine", "tiled", "traveling-salesman"]}, {"tokens": ["mypy", "optional", "static", "typing", "python"], "tags": ["github:python/mypy", "mypy", "Python", "linter", "python", "typechecker", "types", "typing"]}, {"tokens": ["common", "robotics", "utility", "common", "utility", "function", "algorithm", "robotics", "work", "used", "arc", "arm", "lab", "tri"], "tags": ["github:calderpg/common_robotics_utilities", "common_robotics_utilities", "C++"]}, {"tokens": ["drake", "planning", "control", "analysis", "toolbox", "nonlinear", "dynamical", "system", "please", "see", "wiki", "documentation"], "tags": ["github:mwoehlke-kitware/drake", "drake", "C++"]}, {"tokens": ["drake", "planning", "control", "analysis", "toolbox", "nonlinear", "dynamical", "system", "info"], "tags": ["github:jwnimmer-tri/drake", "drake", "C++"]}, {"tokens": ["pybind", "seamless", "operability", "python"], "tags": ["github:mwoehlke-kitware/pybind11", "pybind11", "C++"]}, {"tokens": ["readosm", "readosm", "open", "source", "library", "extract", "valid", "data", "within", "open", "street", "map", "input", "file", "git", "repository", "fork"], "tags": ["github:mwoehlke-kitware/readosm", "readosm", "Shell"]}], "github:hardikparwana": [{"tokens": ["px", "quad", "vicon", "starter", "starter", "code", "running", "px", "quadrotor", "vicon", "support", "built"], "tags": ["github:dasc-lab/px4-quad-vicon-starter", "px4-quad-vicon-starter", "C++"]}, {"tokens": ["px", "ro", "com", "ro", "ro", "interface", "px", "fast", "rtps", "bridge"], "tags": ["github:dasc-lab/px4_ros_com", "px4_ros_com", "Python"]}, {"tokens": ["px", "ro", "starter", "ro", "package", "controlling", "px", "external", "controller"], "tags": ["github:hardikparwana/px4-ros2-starter", "px4-ros2-starter", "C++"]}, {"tokens": ["segway", "sim", "ro", "based", "segway", "simulation", "environment", "includes", "branch", "mpc", "control", "barrier", "function", "new", "version", "cyberpod", "sim", "ro"], "tags": ["github:hardikparwana/segway_sim", "segway_sim", "C++"]}, {"tokens": ["rover", "driver", "package", "controlling", "rover", "ro", "remotely"], "tags": ["github:dasc-lab/rover-driver", "rover-driver", "Python"]}, {"tokens": ["crazyswarm", "large", "quadcopter", "swarm"], "tags": ["github:dasc-lab/crazyswarm", "crazyswarm", "Python"]}], "github:tautme": [{"tokens": ["aa", "dc", "foundation", "astronomical", "data", "science", "aa"], "tags": ["github:tautme/2022-06-11-aas-dc", "2022-06-11-aas-dc", "Jupyter Notebook"]}, {"tokens": ["carpentry", "org", "carpentry", "website"], "tags": ["github:carpentries/carpentries.org", "carpentries.org", "HTML"]}, {"tokens": ["li", "next", "generation", "serverless", "crawler", "covid", "data"], "tags": ["github:covidatlas/li", "li", "JavaScript"]}, {"tokens": ["workshop", "template", "rendering", "workshop", "template", "site"], "tags": ["github:tautme/workshop-template", "workshop-template", "HTML"]}, {"tokens": ["corona", "bubble", "count", "case", "covid", "mile", "radius", "around"], "tags": ["github:tautme/corona-bubble-in-R", "corona-bubble-in-R", "R"]}, {"tokens": ["stsci", "online", "software", "carpentry", "shell", "git", "plotting", "programming", "python", "workshop", "space", "telescope", "science", "institute", "october"], "tags": ["github:tautme/2021-10-18-stsci-online", "2021-10-18-stsci-online", "HTML"]}, {"tokens": ["facilitation", "leading", "practice", "repository", "associated", "website", "serf", "official", "home", "leading", "practice", "facilitation", "developed", "participant", "nsf", "aci", "ref", "project"], "tags": ["github:tautme/facilitation_leading_practices", "facilitation_leading_practices", "HTML"]}, {"tokens": ["uark", "method", "software", "demo", "method", "graduate", "student", "meeting"], "tags": ["github:tautme/2020-12-04-uark-methods", "2020-12-04-uark-methods", "R"]}, {"tokens": ["hackerrank", "code", "challenge"], "tags": ["github:tautme/hackerrank", "hackerrank", "Python"]}, {"tokens": ["python", "novice", "gapminder", "plotting", "programming", "python"], "tags": ["github:tautme/python-novice-gapminder", "python-novice-gapminder", "HTML"]}, {"tokens": ["amplify", "amplify", "console", "ci", "cd", "demo"], "tags": ["github:tautme/amplify", "amplify", "JavaScript"]}, {"tokens": ["fayetteville", "learn", "fayetteville", "online"], "tags": ["github:tautme/2020-06-27-R-Fayetteville", "2020-06-27-R-Fayetteville", "HTML", "data", "r", "software"]}, {"tokens": ["coronadatascraper", "covid", "coronavirus", "data", "scraped", "government", "curated", "data", "source"], "tags": ["github:tautme/coronadatascraper", "coronadatascraper", "JavaScript"]}, {"tokens": ["carpentry", "instructor", "checklist", "checklist", "carpentry", "instructor", "prepping", "workshop"], "tags": ["github:tautme/carpentries_instructor_checklist", "carpentries_instructor_checklist", "HTML"]}], "github:ZeroZ-lab": [{"tokens": ["next", "j", "react", "framework"], "tags": ["github:ZeroZ-lab/next.js", "next.js", "JavaScript"]}], "github:SwetaNikki": [{"tokens": ["regression", "tf", "pick", "choose", "one", "regression", "data", "set", "form", "link", "create", "end", "end", "model", "using", "tensorflow", "api", "passion", "assignment", "need", "problem", "discus", "class", "anyhow", "repo", "link", "data"], "tags": ["github:SwetaNikki/Regression-by-TF2", "Regression-by-TF2", "Jupyter Notebook"]}, {"tokens": ["facial", "expression", "using", "kera", "facial", "expression", "recognition", "kera", "develop", "facial", "expression", "recognition", "model", "kera", "build", "train", "convolutional", "neural", "network", "cnn", "apply", "model", "real", "time", "video", "stream", "image", "data"], "tags": ["github:SwetaNikki/Facial-Expression-using-Keras", "Facial-Expression-using-Keras", "Python"]}, {"tokens": ["face", "mask", "detection", "kera", "detection", "mask"], "tags": ["github:SwetaNikki/MaskDetection", "Face-Mask-Detection-Keras", "Jupyter Notebook"]}], "github:versha-khare": [{"tokens": ["word", "statistic", "repo", "project", "perform", "basic", "statistic", "word", "given", "document", "string"], "tags": ["github:versha-khare/Word_Statistics", "Word_Statistics", "Java"]}, {"tokens": ["opencv", "open", "source", "computer", "vision", "library"], "tags": ["github:versha-khare/opencv", "opencv", "C++"]}], "github:iCounterBOX": [{"tokens": ["esp", "wifi", "sniff", "ble", "android", "app", "pathfinderle", "esp", "wifi", "sniffer", "connected", "via", "ble", "android", "app"], "tags": ["github:iCounterBOX/ESP32_WiFi_SNIFF_BLE_ANDROID-APP_PathFinderLE_V01", "ESP32_WiFi_SNIFF_BLE_ANDROID-APP_PathFinderLE_V01", "C++"]}], "github:shudingbo": [{"tokens": ["egg", "cache", "cache", "plugin", "eggjs"], "tags": ["github:hexindai/egg-cache", "egg-cache", "JavaScript", "egg-cache", "egg-plugin", "eggjs"]}, {"tokens": ["ant", "design", "vue", "enterprise", "class", "ui", "component", "based", "ant", "design", "vue"], "tags": ["github:vueComponent/ant-design-vue", "ant-design-vue", "Vue", "ant", "ant-design", "antd", "antd-vue", "css", "enterprise", "frontend", "react", "ui", "ui-design", "vue", "vue-antd", "vue-antd-ui", "vue3", "vuecomponent"]}, {"tokens": ["sex", "pomelo", "fast", "scalable", "distributed", "game", "server", "framework", "node", "j"], "tags": ["github:sex-pomelo/sex-pomelo", "sex-pomelo", "JavaScript", "pomelo", "sex-pomelo"]}, {"tokens": ["sex", "pomelo", "rpc", "rpc", "framework", "pomelo"], "tags": ["github:sex-pomelo/sex-pomelo-rpc", "sex-pomelo-rpc", "JavaScript"]}, {"tokens": ["sex", "pomelo", "scheduler", "high", "performance", "schedule", "module", "calling", "scheduled", "task"], "tags": ["github:sex-pomelo/sex-pomelo-scheduler", "sex-pomelo-scheduler", "JavaScript"]}, {"tokens": ["sex", "chatofpomelo", "websocket", "pomelo", "chat", "example"], "tags": ["github:sex-pomelo/sex-chatofpomelo-websocket", "sex-chatofpomelo-websocket", "JavaScript"]}, {"tokens": ["myslot", "myslot", "content", "mana"], "tags": ["github:shudingbo/myslot", "myslot", "HTML"]}, {"tokens": ["sex", "scale", "plugin", "server", "auto", "start", "plugin"], "tags": ["github:sex-pomelo/sex-scale-plugin", "sex-scale-plugin", "JavaScript"]}, {"tokens": ["egg", "cache", "cache", "plugin", "eggjs"], "tags": ["github:shudingbo/egg-cache", "egg-cache", "JavaScript"]}, {"tokens": ["sex", "pomelo", "monitor", "monitor", "operating", "system", "info", "process", "info"], "tags": ["github:sex-pomelo/sex-pomelo-monitor", "sex-pomelo-monitor", "JavaScript"]}, {"tokens": ["node", "ipaddrbase", "ip", "addr", "base", "innocence", "ip", "address", "library", "reset", "version"], "tags": ["github:shudingbo/node-ipaddrbase", "node-ipaddrbase", "JavaScript", "ip", "ipaddr"]}, {"tokens": ["sex", "pomelo", "web", "web", "pomelo", "admin", "sex", "pomelo"], "tags": ["github:shudingbo/sex-pomelo-web", "sex-pomelo-web", "Vue"]}, {"tokens": ["egg", "pomelo", "pomelo", "cli", "egg"], "tags": ["github:shudingbo/egg-pomelo", "egg-pomelo", "JavaScript"]}, {"tokens": ["sex", "pomelo", "admin", "admin", "module", "pomelo", "monitor", "system"], "tags": ["github:sex-pomelo/sex-pomelo-admin", "sex-pomelo-admin", "JavaScript"]}, {"tokens": ["pomelo", "pomelo", "plugin"], "tags": ["github:sex-pomelo/pomelo-i18n", "pomelo-i18n", "JavaScript", "i18n", "pomelo"]}, {"tokens": ["node", "cppmsg", "parse", "binary", "data", "also", "generate", "binary", "data", "json", "object", "phrase"], "tags": ["github:shudingbo/node-cppMsg", "node-cppMsg", "JavaScript", "binary-structure", "c-struct", "cpp-memory-layouts", "struct"]}, {"tokens": ["clifflite", "cli", "output", "formatting", "tool"], "tags": ["github:shudingbo/clifflite", "clifflite", "JavaScript"]}, {"tokens": ["sex", "pomelo", "logger", "logger", "wrapper", "logjs", "pomelo", "logger", "system"], "tags": ["github:sex-pomelo/sex-pomelo-logger", "sex-pomelo-logger", "JavaScript"]}], "github:Ouyang-Zhaoxing": [{"tokens": ["substrate", "comprehensive", "net", "sdk", "reading", "writing", "minecraft", "world", "data"], "tags": ["github:minecraft-dotnet/Substrate", "Substrate", "C#"]}, {"tokens": ["drcobj", "dracothree", "j", "use", "draco", "compress", "decompress", "three", "j", "model", "file"], "tags": ["github:Ouyang-Zhaoxing/drcobj", "drcobj", "JavaScript", "3d", "draco", "draco-threejs", "draco3d", "game-model", "html-3d", "html-canvas", "html5-game", "model", "model-format", "model-loader", "three-js", "threejs", "threejs-model"]}], "github:davidmartos96": [{"tokens": ["flutter", "device", "preview", "approximate", "app", "look", "performs", "another", "device"], "tags": ["github:aloisdeniel/flutter_device_preview", "flutter_device_preview", "Dart", "device", "flutter", "preview", "tool"]}, {"tokens": ["bottle", "run", "window", "software", "game", "linux"], "tags": ["github:bottlesdevs/Bottles", "Bottles", "Python", "appimage", "archlinux", "bottles", "bottles-versioning", "dxvk", "games", "gaming", "linux", "msi", "ppa", "proton", "python", "wine", "wine-prefixes", "wineprefix", "wineprefix-manager"]}, {"tokens": ["postgres", "pool", "postgresql", "connection", "pool", "wide", "range", "automated", "reconnect", "parameter", "dart"], "tags": ["github:agilord/postgres_pool", "postgres_pool", "Dart"]}, {"tokens": ["sqflite", "sqlcipher", "sqlite", "flutter", "plugin"], "tags": ["github:davidmartos96/sqflite_sqlcipher", "sqflite_sqlcipher", "Dart"]}, {"tokens": ["magic", "preconstructed", "deck", "magic", "gathering", "preconstructed", "deck"], "tags": ["github:taw/magic-preconstructed-decks", "magic-preconstructed-decks", "Ruby"]}, {"tokens": ["fvm", "flutter", "version", "management", "simple", "cli", "manage", "flutter", "sdk", "version"], "tags": ["github:leoafarias/fvm", "fvm", "Dart", "cli", "dart", "flutter", "flutter-releases", "flutter-sdk-versions", "fvm", "sdk"]}, {"tokens": ["riverpod", "reactive", "caching", "data", "binding", "framework", "riverpod", "make", "working", "asynchronous", "code", "breeze"], "tags": ["github:rrousselGit/river_pod", "riverpod", "Dart", "dart", "flutter", "hacktoberfest", "inheritedwidget", "provider", "riverpod", "service-locator", "state-management"]}, {"tokens": ["sqflite", "sqlite", "flutter", "plugin"], "tags": ["github:tekartik/sqflite", "sqflite", "Dart"]}, {"tokens": ["docker", "image", "flutter", "docker", "image", "flutter"], "tags": ["github:cirruslabs/docker-images-flutter", "docker-images-flutter", "Shell"]}, {"tokens": ["sqlite", "dart", "sqlite", "binding", "dart"], "tags": ["github:simolus3/sqlite3.dart", "sqlite3.dart", "Dart"]}, {"tokens": ["flutter", "cached", "network", "image", "download", "cache", "show", "image", "flutter", "app"], "tags": ["github:Baseflow/flutter_cached_network_image", "flutter_cached_network_image", "Dart", "baseflow", "caching", "flutter", "image", "imageprovider"]}, {"tokens": ["appwrite", "backend", "minus", "hassle"], "tags": ["github:appwrite/appwrite", "appwrite", "TypeScript", "android", "appwrite", "backend", "backend-as-a-service", "docker", "firebase", "flutter", "hacktoberfest", "ios", "javascript", "nextjs", "react", "react-native", "reactnative", "self-hosted", "selfhosted", "serverless", "swift", "web"]}, {"tokens": ["flutter", "speed", "dial", "flutter", "plugin", "implement", "material", "design", "speed", "dial"], "tags": ["github:darioielardi/flutter_speed_dial", "flutter_speed_dial", "Dart", "flutter", "flutter-plugin", "material-design"]}, {"tokens": ["go", "flutter", "flutter", "window", "macos", "linux", "based", "flutter", "embedding", "go", "glfw"], "tags": ["github:go-flutter-desktop/go-flutter", "go-flutter", "Go", "cross-platform", "desktop", "flutter", "glfw", "go", "golang", "hover", "linux", "macos", "ui", "windows"]}, {"tokens": ["libnut", "core", "node", "api", "addon", "desktop", "automation"], "tags": ["github:nut-tree/libnut", "libnut-core", "C"]}, {"tokens": ["postgresql", "dart", "dart", "postgresql", "driver", "support", "extended", "query", "format", "binary", "protocol", "statement", "reuse"], "tags": ["github:isoos/postgresql-dart", "postgresql-dart", "Dart"]}, {"tokens": ["catcher", "flutter", "error", "catching", "handling", "plugin", "handle", "report", "exception", "app"], "tags": ["github:jhomlala/catcher", "catcher", "Dart", "android", "dart", "dartlang", "flutter", "flutter-examples", "flutter-package", "flutter-plugin", "ios"]}, {"tokens": ["yumex", "dnf", "old", "next", "generation", "yum", "extender", "using", "dnf", "backend"], "tags": ["github:timlau/yumex-dnf", "yumex-dnf-old", "Python"]}, {"tokens": ["countdown", "countdown", "countdown", "pause", "resume", "control"], "tags": ["github:BenNG/countdown", "countdown", "Dart"]}, {"tokens": ["kt", "dart", "port", "kotlin", "stdlib", "dart", "flutter", "including", "immutable", "collection", "ktlist", "ktmap", "ktset", "package"], "tags": ["github:passsy/kt.dart", "kt.dart", "Dart", "collection", "dart", "dartlang", "flutter", "hacktoberfest", "kotlin"]}, {"tokens": ["fading", "edge", "scrollview", "flutter", "library", "displaying", "fading", "edge", "scroll", "view"], "tags": ["github:mponkin/fading_edge_scrollview", "fading_edge_scrollview", "Dart"]}, {"tokens": ["serializr", "serialize", "deserialize", "complex", "object", "graph", "json", "javascript", "class"], "tags": ["github:mobxjs/serializr", "serializr", "TypeScript"]}, {"tokens": ["docker", "image", "flutter", "docker", "image", "flutter"], "tags": ["github:davidmartos96/docker-images-flutter", "docker-images-flutter", "Shell"]}, {"tokens": ["vscode", "versionlens", "show", "version", "information", "package", "using", "code", "lens"], "tags": ["github:davidmartos96/vscode-versionlens", "vscode-versionlens", "JavaScript"]}, {"tokens": ["postgres", "pool", "postgresql", "connection", "pool", "wide", "range", "automated", "reconnect", "parameter", "dart"], "tags": ["github:davidmartos96/postgres_pool", "postgres_pool", "Dart"]}, {"tokens": ["magic", "preconstructed", "deck", "magic", "gathering", "preconstructed", "deck"], "tags": ["github:davidmartos96/magic-preconstructed-decks", "magic-preconstructed-decks", "Ruby"]}, {"tokens": ["fvm", "flutter", "version", "management", "simple", "cli", "manage", "flutter", "sdk", "version", "per", "project"], "tags": ["github:davidmartos96/fvm", "fvm", "Dart"]}, {"tokens": ["go", "flutter", "plugin", "sqlite", "try", "implement", "go", "version", "sqlite", "channel", "flutter", "sqflite", "plugin", "desktop"], "tags": ["github:davidmartos96/go-flutter-plugin-sqlite", "go-flutter-plugin-sqlite", "Go"]}, {"tokens": ["admob", "flutter", "admob", "flutter", "plugin", "show", "banner", "ad", "using", "native", "platform", "view"], "tags": ["github:js2702/admob_flutter", "admob_flutter", "Swift"]}, {"tokens": ["go", "flutter", "flutter", "window", "macos", "linux", "based", "flutter", "embedding", "go", "glfw"], "tags": ["github:davidmartos96/go-flutter", "go-flutter", "Go"]}, {"tokens": ["serializr", "serialize", "deserialize", "complex", "object", "graph", "json", "javascript", "class"], "tags": ["github:davidmartos96/serializr", "serializr", "TypeScript"]}, {"tokens": ["libnut", "node", "api", "addon", "desktop", "automation"], "tags": ["github:davidmartos96/libnut", "libnut", "C"]}, {"tokens": ["countdown", "countdown", "countdown", "pause", "resume", "control"], "tags": ["github:davidmartos96/countdown", "countdown", "Dart"]}, {"tokens": ["fading", "edge", "scrollview", "flutter", "library", "displaying", "fading", "edge", "scroll", "view"], "tags": ["github:davidmartos96/fading_edge_scrollview", "fading_edge_scrollview", "Dart"]}, {"tokens": ["go", "router", "bottom", "nav", "demo", "persistent", "bottom", "navigation", "demo", "go", "router"], "tags": ["github:davidmartos96/go_router_bottom_nav_demo", "go_router_bottom_nav_demo", "C++"]}, {"tokens": ["flutter", "draggable", "scrollbar", "draggable", "scrollbar", "scrollbar", "dragged", "quickly", "navigation", "vertical", "list", "additional", "option", "showing", "label", "next", "scrollthumb", "information", "current", "item", "maintainer", "marica"], "tags": ["github:davidmartos96/flutter-draggable-scrollbar", "flutter-draggable-scrollbar", "Dart"]}, {"tokens": ["flutter", "installer", "info", "installer", "information", "flutter", "apps"], "tags": ["github:davidmartos96/flutter-installer-info", "flutter-installer-info", "Dart"]}, {"tokens": ["dart", "junitreport", "application", "generate", "junit", "xml", "report", "dart", "test", "run"], "tags": ["github:davidmartos96/dart-junitreport", "dart-junitreport", "Dart"]}, {"tokens": ["flutter", "speed", "dial", "flutter", "plugin", "implement", "material", "design", "speed", "dial"], "tags": ["github:davidmartos96/flutter_speed_dial", "flutter_speed_dial", "Dart"]}, {"tokens": ["flutter", "city", "hack", "flutter", "project"], "tags": ["github:davidmartos96/flutter_city", "flutter_city", "Dart"]}], "github:sharathguvvala": [{"tokens": ["interventus", "social", "medium", "web", "application"], "tags": ["github:sharathguvvala/Interventus", "Interventus", "JavaScript"]}, {"tokens": ["chatter", "web", "chatting", "application", "using", "socket", "io"], "tags": ["github:sharathguvvala/Chatter", "Chatter", "JavaScript", "nodejs", "socket-io"]}], "github:chanibal": [{"tokens": ["free", "programming", "book", "book", "freely", "available", "programming", "book"], "tags": ["github:EbookFoundation/free-programming-books", "free-programming-books", "HTML", "books", "education", "hacktoberfest", "list", "resource"]}, {"tokens": ["webplayer", "easiest", "way", "stream", "video", "picture", "website", "blog"], "tags": ["github:BIVROST/360WebPlayer", "360WebPlayer", "JavaScript", "360-photo", "360-video", "cardboard", "hacktoberfest", "hls", "video-players", "webvr", "webxr"]}, {"tokens": ["beginnerdirectd", "starter", "code", "sample", "directd"], "tags": ["github:kevinmoran/BeginnerDirect3D11", "BeginnerDirect3D11", "C", "3d-graphics", "direct3d", "direct3d11", "directx", "directx-11", "game-development", "gamedev", "graphics", "graphics-programming", "graphics-rendering", "rendering", "rendering-3d-graphics"]}, {"tokens": ["deno", "sqlite", "deno", "sqlite", "module"], "tags": ["github:dyedgreen/deno-sqlite", "deno-sqlite", "TypeScript", "browser", "database", "deno", "deno-sqlite", "docs-deno", "javascript", "sqlite3", "typescript", "wasm", "webassembly"]}, {"tokens": ["playerwindows", "video", "player", "immersive", "spherical", "video", "let", "play", "virtual", "reality", "video", "pc", "using", "vr", "headset"], "tags": ["github:BIVROST/360PlayerWindows", "360PlayerWindows", "C#"]}, {"tokens": ["logcxx", "v", "helper", "use", "logcxx", "visual", "studio", "toolkit"], "tags": ["github:chanibal/log4cxx-vs142", "log4cxx-vs142", "C"]}, {"tokens": ["timesheet", "tool", "simple", "time", "tracking", "tool", "parses", "timesheets", "show", "summary"], "tags": ["github:chanibal/timesheet-tool", "timesheet-tool", "JavaScript"]}, {"tokens": ["webplayer", "wordpress", "bivrost", "webplayer", "wordpress", "easiest", "way", "stream", "video", "picture", "website", "blog"], "tags": ["github:BIVROST/360WebPlayer-for-WordPress", "360WebPlayer-for-WordPress", "PHP"]}], "github:luugiathuy": [{"tokens": ["dotfiles", "dotfiles", "including", "zsh", "prezto", "vim"], "tags": ["github:luugiathuy/dotfiles", "dotfiles", "Vim Script", "dotfiles", "tmux", "vim", "zprezto", "zsh"]}, {"tokens": ["prezto", "configuration", "framework", "zsh"], "tags": ["github:luugiathuy/prezto", "prezto", "Shell"]}, {"tokens": ["reversigame", "reversi", "othello", "game", "written", "java"], "tags": ["github:luugiathuy/ReversiGame", "ReversiGame", "Java"]}], "github:build-1000": [{"tokens": ["clusterduck", "protocol", "firmware", "ad", "hoc", "mesh", "network", "internet", "thing", "device", "based", "lora", "long", "range", "radio", "deployed", "quickly", "low", "cost"], "tags": ["github:Call-for-Code/ClusterDuck-Protocol", "ClusterDuck-Protocol", "C++", "call-for-code", "ducks", "emergency-network", "hacktoberfest", "iot", "lora", "mesh", "owl"]}], "github:dragosbus": [{"tokens": ["scoreboard", "react", "project", "react", "basic", "teamtreehouse"], "tags": ["github:dragosbus/scoreboard-react", "scoreboard-react", "JavaScript"]}], "github:K-ran": [{"tokens": ["personalreactsite", "personal", "website", "code", "written", "react"], "tags": ["github:K-ran/personalReactSite", "personalReactSite", "JavaScript"]}, {"tokens": ["consoleimagegenerator", "application", "display", "image", "video", "file", "console"], "tags": ["github:K-ran/ConsoleImageGenerator", "ConsoleImageGenerator", "CMake"]}, {"tokens": ["dicechesssite", "front", "dice", "chess", "server", "written", "react"], "tags": ["github:K-ran/dicechesssite", "dicechesssite", "JavaScript"]}, {"tokens": ["dicechessdicegoserver", "go", "server", "dice", "chess", "dice"], "tags": ["github:K-ran/diceChessDiceGoServer", "diceChessDiceGoServer", "Go"]}], "github:Aoun-Hussain": [{"tokens": ["sleep", "scheduling", "routing", "protocol", "water", "wireless", "sensor", "network", "using", "milp", "repository", "cited", "research", "paper", "titled", "energy", "efficient", "approach", "oil", "detection", "smart", "ocean", "published", "ieee", "sensor", "journal", "oct"], "tags": ["github:Aoun-Hussain/Sleep-Scheduling-Routing-Protocol-for-Under-water-Wireless-Sensor-Networks-Using-MILP", "Sleep-Scheduling-Routing-Protocol-for-Under-water-Wireless-Sensor-Networks-Using-MILP", "Python"]}, {"tokens": ["data", "set", "asl", "psl", "gesture", "repository", "consists", "red", "channeled", "bit", "wise", "extracted", "gesture", "asl", "psl", "made", "specialized", "data", "set", "train", "various", "ml", "classifier", "svm", "cnn", "alexnet", "vgg"], "tags": ["github:Aoun-Hussain/Data-Set-for-ASL-and-PSL-Gestures", "Data-Set-for-ASL-and-PSL-Gestures", "Python"]}, {"tokens": ["deployment", "optimization", "wireless", "sensor", "network", "using", "genetic", "algorithm", "repository", "contains", "code", "network", "simulation", "used", "paper", "evolution", "based", "deployment", "scheme", "green", "internet", "thing", "sleep", "scheduling", "oil", "detection", "routing", "protocol", "smart", "ocean", "using", "internet", "thing", "paper", "accepted", "published", "ieee", "wf", "iot", "ieee", "wf", "iot"], "tags": ["github:Aoun-Hussain/Sleep-Scheduling-Routing-Protocol-for-Under-water-Wireless-Sensor-Networks-Using-GA", "Deployment-Optimization-for-Wireless-Sensor-Networks-Using-Genetic-Algorithm", "Python"]}, {"tokens": ["fault", "tolerant", "file", "system", "efficient", "distributed", "file", "system", "improves", "performance", "input", "output", "operation", "file", "logging", "proposes", "data", "persistent", "system", "recovered", "data", "lost"], "tags": ["github:Aoun-Hussain/Fault-Tolerant-File-System", "Fault-Tolerant-File-System", "C++"]}, {"tokens": ["pipelined", "risc", "processor", "public", "repository", "pipelined", "risc", "processor", "capable", "executing", "bubble", "sort", "processor", "consists", "bit", "alu", "necessary", "forwarding", "circuitry", "hazard", "detection", "unit", "pipeline", "flushing", "detect", "control", "data", "hazard", "program"], "tags": ["github:Aoun-Hussain/A-Pipelined-RISC-V-Processor", "A-Pipelined-RISC-V-Processor", "Verilog"]}, {"tokens": ["drug", "discovery", "using", "gan", "model", "implement", "deep", "learning", "reinforcement", "learning", "approach", "integrates", "two", "different", "deep", "neural", "network", "generator", "discriminator", "reinforcement", "learning", "two", "model", "trained", "separately", "used", "jointly", "generate", "novel", "targeted", "chemical", "library"], "tags": ["github:Aoun-Hussain/Drug-Discovery-Using-GAN", "Drug-Discovery-Using-GAN", "Python"]}, {"tokens": ["deepqlearning", "using", "tf", "implementation", "deep", "learning", "technique", "using", "two", "neural", "network", "cnn", "ann", "rl", "lunar", "lander", "testing", "performance", "tensor", "flow", "used", "implementation"], "tags": ["github:Aoun-Hussain/DeepQLearning-Using-TF", "DeepQLearning-Using-TF", "Python"]}, {"tokens": ["type", "gans", "repository", "consists", "notebook", "various", "type", "gans", "implemented", "stanford", "car", "dataset", "result", "analyzed"], "tags": ["github:Aoun-Hussain/Types-of-GANs", "Types-of-GANs", "Jupyter Notebook"]}, {"tokens": ["socket", "communication", "using", "sfml", "project", "consist", "client", "server", "file", "us", "sfml", "library", "socket", "communication"], "tags": ["github:Aoun-Hussain/Socket-Communication-Using-SFML", "Socket-Communication-Using-SFML", "C++"]}, {"tokens": ["opengl", "simulation", "uav", "football", "field", "project", "consists", "file", "contribute", "simulation", "unmanned", "aerial", "vehicle", "football", "field", "us", "opengl"], "tags": ["github:Aoun-Hussain/OpenGL-Simulation-UAV-Football-Field", "OpenGL-Simulation-UAV-Football-Field", "C++"]}, {"tokens": ["deployment", "optimization", "wireless", "sensor", "network", "using", "mixed", "integer", "linear", "programming", "repository", "contains", "code", "network", "simulation", "used", "paper", "evolution", "based", "deployment", "scheme", "green", "internet", "thing", "sleep", "scheduling", "oil", "detection", "routing", "protocol", "smart", "ocean", "using", "internet", "thing", "paper", "accepted", "published", "ieee", "wf", "iot", "ieee", "wf", "iot"], "tags": ["github:Aoun-Hussain/Deployment-Optimization-for-Wireless-Sensor-Networks-Using-Mixed-Integer-Linear-Programming", "Deployment-Optimization-for-Wireless-Sensor-Networks-Using-Mixed-Integer-Linear-Programming", "Python"]}, {"tokens": ["ml", "classifier", "gesture", "recognition", "asl", "psl", "repository", "consists", "code", "ml", "classifier", "used", "gesture", "recognition", "asl", "psl"], "tags": ["github:Aoun-Hussain/ML-Classifiers-for-Gesture-Recognition-of-ASL-and-PSL", "ML-Classifiers-for-Gesture-Recognition-of-ASL-and-PSL", "Jupyter Notebook"]}, {"tokens": ["machine", "learning", "technique", "malaria", "cifar", "datasets", "repository", "consists", "google", "colab", "notebook", "notebook", "consists", "various", "machine", "learning", "deep", "learning", "model", "applied", "malaria", "cifar", "datasets"], "tags": ["github:Aoun-Hussain/Machine-Learning-Techniques-on-The-Malaria-and-Cifar10-Datasets", "Machine-Learning-Techniques-on-The-Malaria-and-Cifar10-Datasets", "Jupyter Notebook"]}, {"tokens": ["data", "management", "interface", "data", "management", "interface", "used", "research", "project", "water", "availability", "pakistan", "project", "funded", "higher", "education", "commision", "hec", "pakistan", "interface", "built", "python", "using", "object", "oriented", "programming", "manages", "data", "sent", "multiple", "water", "flowmeters", "gsm", "protocol", "using", "simc", "gsm", "module", "sending", "receiving", "water", "reading"], "tags": ["github:Aoun-Hussain/Data-Management-Interface", "Data-Management-Interface", "Python"]}, {"tokens": ["transform", "raw", "data", "nested", "json", "repository", "consists", "python", "script", "read", "raw", "data", "file", "txt", "format", "transforms", "product", "record", "output", "clean", "nested", "json", "file"], "tags": ["github:Aoun-Hussain/Transform-Raw-Data-To-Nested-Json", "Transform-Raw-Data-To-Nested-JSON", "Python"]}, {"tokens": ["pehelwan", "liberal", "goth", "cleanup", "game", "made", "using", "sdl", "library", "object", "oriented", "programming"], "tags": ["github:Aoun-Hussain/Pehelwan-Liberal-Goth-Cleanup", "Pehelwan-Liberal-Goth-Cleanup", "C++"]}, {"tokens": ["deep", "learning", "ecg", "repository", "consists", "file", "notebook", "implement", "traditional", "machine", "learning", "modern", "deep", "learning", "technique", "prediction", "using", "ecg", "hr"], "tags": ["github:Aoun-Hussain/Deep-Learning-on-ECG", "Deep-Learning-on-ECG", "Jupyter Notebook"]}, {"tokens": ["commerce", "auto", "cropping", "model", "using", "machine", "learning", "repository", "consists", "python", "code", "take", "input", "raw", "unedited", "image", "male", "female", "model", "facing", "front", "output", "three", "image", "focusing", "upper", "body", "lower", "body", "full", "body", "output", "carefully", "cropped", "focus", "product", "model", "wearing", "making", "code", "suitable", "commerce", "website", "reduces", "manual", "labour", "editing", "thousand", "picture"], "tags": ["github:Aoun-Hussain/Ecommerce-Auto-Cropping-Model-Using-Machine-Learning", "E-Commerce-Auto-Cropping-Model-Using-Machine-Learning", "Python"]}, {"tokens": ["openmpi", "multi", "threading", "communication", "openmpi", "project", "main", "cpp", "take", "argument", "following", "number", "dimension", "following", "total", "iteration", "executes", "laplace", "equation", "using", "jacobian", "iteration", "update", "temperature", "square", "plate", "writes", "output", "temperature", "coordinate", "csv", "file", "output", "total", "execution", "time", "us", "openmpi", "multi", "threaded", "processor", "purpose"], "tags": ["github:Aoun-Hussain/OpenMPI-Multi-Threading-Communication", "OpenMPI-Multi-Threading-Communication", "C++"]}, {"tokens": ["cuda", "parallel", "computing", "project", "consists", "cuda", "source", "file", "take", "argument", "following", "number", "dimension", "following", "total", "iteration", "executes", "laplace", "equation", "using", "jacobian", "iteration", "update", "temperature", "square", "plate", "writes", "output", "temperature", "coordinate", "csv", "file", "output", "total", "execution", "time", "us", "cuda", "multi", "threaded", "gpus", "purpose"], "tags": ["github:Aoun-Hussain/CUDA-Parallel-Computing", "CUDA-Parallel-Computing", "Cuda"]}, {"tokens": ["laser", "reflection", "simulation", "project", "consists", "cpp", "source", "file", "solution", "take", "single", "command", "line", "argument", "represents", "initial", "reflection", "location", "along", "ab", "segment", "calculates", "number", "time", "beam", "reflected", "internal", "surface", "white", "cell", "exiting"], "tags": ["github:Aoun-Hussain/Laser-Reflection-Simulation", "Laser-Reflection-Simulation", "C++"]}, {"tokens": ["langton", "ant", "simulation", "project", "consists", "cpp", "source", "file", "output", "number", "black", "square", "left", "white", "square", "grid", "text", "file", "ant", "take", "given", "number", "move", "according", "standard", "rule", "define", "langton", "ant", "problem"], "tags": ["github:Aoun-Hussain/Langton-s-Ant-Simulation", "Langton-s-Ant-Simulation", "C++"]}, {"tokens": ["prime", "factor", "file", "writing", "project", "consists", "cpp", "source", "file", "check", "valid", "invalid", "command", "line", "input", "file", "return", "string", "prime", "factor", "number", "argument", "separated", "comma"], "tags": ["github:Aoun-Hussain/Prime-Factors-And-File-Writing", "Prime-Factors-And-File-Writing", "C++"]}], "github:tehokang": [{"tokens": ["sedec", "java", "advanced", "version", "sedec", "support", "decoding", "encoding", "mpeg", "section", "like", "mpeg", "dvb"], "tags": ["github:tehokang/sedec-2.0-java", "sedec-2.0-java", "Java", "dvb", "mpeg", "mpeg-descriptor", "mpeg-h", "mpeg-section", "mpeg-ts"]}], "github:angusivory": [{"tokens": ["morse", "code", "arduino", "project", "convert", "morse", "code", "button", "press", "character"], "tags": ["github:angusivory/morse-code", "morse-code", "C++"]}, {"tokens": ["altitude", "calculator", "input", "desired", "altitude", "temperature", "pressure", "density", "calculated", "height"], "tags": ["github:angusivory/altitude-calculator", "altitude-calculator", "Python"]}], "github:pnbao": [{"tokens": ["jbbmo", "introduction", "git", "github", "introduction", "git", "github"], "tags": ["github:ibm-developer-skills-network/jbbmo-Introduction-to-Git-and-GitHub", "jbbmo-Introduction-to-Git-and-GitHub", "Python"]}, {"tokens": ["react", "linear", "gradient", "picker", "linear", "gradient", "picker", "written", "react"], "tags": ["github:pnbao/react-linear-gradient-picker", "react-linear-gradient-picker", "JavaScript"]}, {"tokens": ["ac", "sso", "server", "single", "sign", "auth", "server", "facebook", "google", "twitter", "logins"], "tags": ["github:pnbao/ac-sso-server", "ac-sso-server", "JavaScript"]}, {"tokens": ["deep", "photo", "enhance", "tensorflow", "implementation", "cvpr", "spotlight", "paper", "deep", "photo", "enhancer", "unpaired", "learning", "image", "enhancement", "photograph", "gans"], "tags": ["github:pnbao/deep-photo-enhance", "deep-photo-enhance", "Python"]}, {"tokens": ["node", "canvas", "lambda", "aws", "lambda", "layer", "node", "canvas"], "tags": ["github:pnbao/node-canvas-lambda", "node-canvas-lambda", "JavaScript"]}], "github:spekulatiusmensch": [{"tokens": ["rcppp", "rust", "programmer"], "tags": ["github:nrc/r4cppp", "r4cppp", "Rust"]}, {"tokens": ["rpi", "wsx", "rust", "rust", "binding", "wsx", "library", "enabled", "bindgen"], "tags": ["github:rpi-ws281x/rpi-ws281x-rust", "rpi-ws281x-rust", "Rust"]}, {"tokens": ["setzer", "latex", "editor", "written", "python", "gtk"], "tags": ["github:cvfosammmm/Setzer", "Setzer", "HTML"]}, {"tokens": ["rpi", "wsx", "rust", "rust", "binding", "wsx", "library", "enabled", "bindgen"], "tags": ["github:spekulatiusmensch/rpi-ws281x-rust", "rpi-ws281x-rust", "Rust"]}], "github:MukunthAG": [{"tokens": ["tictactoe", "minimax", "tictactoe", "game", "using", "minimax", "algorithm"], "tags": ["github:MukunthAG/TicTacToe-Minimax", "TicTacToe-Minimax", "C", "c", "game-theory", "minimax", "tic-tac-toe"]}, {"tokens": ["eye", "posture", "care", "simple", "chrome", "extension", "periodically", "reminds", "relax", "eye", "adjust", "posture"], "tags": ["github:MukunthAG/Eye-and-Posture-Care", "Eye-and-Posture-Care", "JavaScript"]}, {"tokens": ["fourier", "visualized", "tool", "draw", "fourier", "series", "diagram", "given", "image"], "tags": ["github:MukunthAG/Fourier-Visualized", "Fourier-Visualized", "Python"]}], "github:chesha1": [{"tokens": ["seu", "course", "course", "evaluation", "seu", "information", "school"], "tags": ["github:chesha1/seu_course", "seu_course", "Makefile"]}, {"tokens": ["livep", "tool", "process", "live"], "tags": ["github:chesha1/livep", "livep", "Python"]}, {"tokens": ["chesha", "config", "file", "github", "profile"], "tags": ["github:chesha1/chesha1", "chesha1", "config", "github-config"]}, {"tokens": ["stable", "diffusion", "personal", "use", "stable", "diffusion", "sd"], "tags": ["github:chesha1/Stable_Diffusion_v", "stable_diffusion_v", "Python"]}], "github:SrinidhiBalasubramanian": [{"tokens": ["text", "classification", "sanskrit", "using", "rnn", "classify", "sanskrit", "word"], "tags": ["github:SrinidhiBalasubramanian/Text-Classification-Sanskrit", "Text-Classification-Sanskrit", "Jupyter Notebook"]}], "github:Kepler-69c": [{"tokens": ["endless", "sky", "space", "exploration", "trading", "combat", "game"], "tags": ["github:endless-sky/endless-sky", "endless-sky", "C++", "2d-game", "adventure-game", "arcade-game", "c-plus-plus", "cpp", "endless-sky", "exploration", "free", "game", "game-development", "gamedev", "indie", "modding", "multiplatform", "open-source", "opengl", "opensource", "rpg", "space", "top-down"]}, {"tokens": ["kepler", "config", "file", "github", "profile"], "tags": ["github:Kepler-69c/Kepler-69c", "Kepler-69c", "config", "github-config"]}, {"tokens": ["pythonflyby", "simulate", "flyby", "space", "probe", "python"], "tags": ["github:Kepler-69c/pythonFlyby", "pythonFlyby", "Python", "gravity", "gravity-assist", "matplotlib", "python", "python3"]}, {"tokens": ["obsidian", "darkside", "theme", "obsidian"], "tags": ["github:Kepler-69c/obsidian-darkside", "obsidian-darkside", "CSS", "obsidian", "obsidian-theme"]}]}

In [6]:
len(d)

50